In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import gc

# 1. set file path
#### todo: set to path lib

In [2]:
print(os.getcwd())

#notify default path contains \\ by r' with windows sys, / by linux
path = r'G:\py_workspace\ptt\\'
os.listdir(path)

G:\GitHub\anomaly_detection_in_PTT_forum


['.ipynb_checkpoints',
 'bk',
 'Gossiping-34700-39632.json',
 'HatePolitics-0-4062.json',
 'HatePolitics-0-4214.json',
 'HatePolitics-1500-3989.json',
 'notes_crawl.txt',
 'ptt yi fang_20190813_0745.txt',
 'ptt_ana.ipynb',
 'ptt_etl.ipynb',
 'python -m PttWebCrawler -b PublicServan -i 100 200  38889.txt',
 '下載.jpg',
 '下載.png']

# 2. check json format

In [3]:
def check_json_format(file):
    with open(file, 'ab+') as f:
        # read as binary is for reverse file search
        print('check trivial comma in EOF before revesing')
        # * 2 -- end of stream; offset is usually negative
        f.seek(-6, 2)
        byte_clip = f.read(10)
  
        if byte_clip == b'}]},]}' or byte_clip[-4:] == b'}]},':
            #update manual force interupt
            revise_idx = -(byte_clip[::-1].find(b',') + 1)
            print('Revise error json {} ending from comma index {}.'.format(byte_clip, revise_idx))
            #pre -3
            byte_clip = f.seek(revise_idx, 2)
            f.truncate()
            f.write(b']}')        
        else:
            print('Correct json format.')
  
        print('check trivial comma in EOF after revising')
        f.seek(-20, 2)
        print(f.read(200))

In [4]:
b'}]},'[::-1].find(b',') 


#'abcde'[::-1]
#

#'a' in ['b', 'c', 'ba']

0

# 3. parse json file
## notes:
#### 1. bug of special character in nickname
#### ex: rex520368 (b@N9) https://www.ptt.cc/bbs/Lifeismoney/M.1575016206.A.74B.html
#### check method: art_df[art_df.author == 'NoData'] 
#### 2. remove filter feature: (1) no reply (2) unknow author

In [5]:
def ptt_json_to_df(ptt_df):
    art_df = []
    #currently ignore error records
    invalid_lst = []
    
    for x in ptt_df.itertuples():
        #some inconsistent page failed!
        if x.articles == {'error': 'invalid url'}:
            print('invalid record')
            print(x.Index)
            invalid_lst.append(x)
            continue
        art = pd.DataFrame([x.articles])
        
        if art.author[0] is None:
            art.author = 'NoData'    
        else:
            art.author = art.author.replace(r'\([^)]*\)', '',regex=True).values[0].replace(' ', '')
        
        #fixing package typo of columns
        art.rename({'message_conut':'message_count'}, axis=1, inplace=True)
        reply_aggr = pd.DataFrame(art.message_count.values[0], index=[0])
        art.drop(['message_count'], axis=1, inplace=True)
        art = pd.concat([art, reply_aggr], axis=1)
        art_df.append(art)
    art_df = pd.concat(art_df)
    art_df.reset_index(drop=True, inplace=True)
    return art_df

In [6]:
def ptt_article_parser(f_nm):
    ##this parser is for jwlin crawler's result
    print('{} parsing start!'.format(f_nm))
    ptt_df = pd.read_json(f_nm, encoding='utf8')    
    ptt_df_t1 = ptt_json_to_df(ptt_df)
    ptt_df_t1.date = pd.to_datetime(ptt_df_t1.date, format='%a %b %d %H:%M:%S %Y', errors='coerce')
    print('parsing finished!')
    return ptt_df_t1

# 4. reply parsing
## fix this incompleted parser.....

In [7]:
def ptt_reply_parser(art_df):
    ###default check process timestamp on 1/20 articles 
    rpt_seg = 20
    reply_df = []
    rpt_idx = int(art_df.shape[0]/rpt_seg)
    print('Total articles are {}'.format(art_df.shape[0]))

    for idx, x in enumerate(art_df.itertuples()):
        if idx % rpt_idx == 0:
            print(idx)
            print(datetime.now())
            gc.collect()

        if x.messages == []:
            continue
        tmp_reply = pd.DataFrame(x.messages)
        tmp_reply['ip'] = tmp_reply.push_ipdatetime.str.split(' ', n=1, expand=True)[0]
        tmp_ip_dt = tmp_reply.push_ipdatetime.copy()
        tmp_dt = np.where(tmp_ip_dt.str.len() < 20, tmp_ip_dt, tmp_ip_dt.str[-11:])

        #add ip....
        if type(x.date) is not str:
            tmp_reply['reply_datetime'] = str(x.date.year) + '/' + tmp_dt
        else:
            tmp_reply['reply_datetime'] = tmp_dt
        ##以發文日期作為推文的估計值，因為push_ipdatetime的ip 跟 datetime放在一起，有些只有datetime 沒有ip，好像跟RE 有關....
        ##欄位重新命名，push_userid 改為 source

        tmp_reply['author'] = x.author
        tmp_reply['article_id'] = x.article_id
        tmp_reply['board'] = x.board
        tmp_reply['date'] = x.date

        reply_df.append(tmp_reply)
    reply_df = pd.concat(reply_df)
    return reply_df

# 5. intergrate whole function

In [8]:
def intg_parser(path, file_nm):
    exec_dt = datetime.strftime(datetime.now(), '%Y%m%d_%H%M%S')
    file_path = path + file_nm
    check_json_format(file_path)
    art_df = ptt_article_parser(file_path)
    art_df['re_flag'] = np.where(art_df.article_title.str.contains('Re'), 'Y', 'N')
    art_df['expl_flag'] = np.where(art_df['count'] >= 100, 'Y', 'N')
    #avoid comma parse error in csv
    art_df.replace(',', '，', regex=True, inplace=True)

    #selected output article
    slc_col = ['article_id', 'article_title', 'author', 'board', 'content', 'date',
               'ip', 'all', 'boo', 'count', 'neutral', 'push', 're_flag', 'expl_flag']
    #output article csv
    art_df[slc_col].to_csv(path + file_nm.strip('.json') + '_article_{}.csv'.format(exec_dt))

    #parsing reply
    reply_df = ptt_reply_parser(art_df)
    reply_df.reset_index(drop=True, inplace=True)
    reply_df.replace(',', '，', regex=True, inplace=True)
    reply_df[reply_df.push_content.str.contains(',')]

    #not transfer format to datetime(pd.to_datetime(df, errors='coerce')), because there are some elements without year.
    reply_df.reply_datetime.replace('/', '-', regex=True, inplace=True)
    #output reply csv
    reply_df.to_csv(path + file_nm.strip('.json') + '_reply_{}.csv'.format(exec_dt))

# 6. exec whole process
#### todo:  
#### 1. merge file & create file's time range
#### 2. move processed file to backup folder

In [9]:
for file in os.listdir(path):
    if file.endswith('json'):
        print(file)
        intg_parser(path=path, file_nm=file)

Gossiping-34700-39632.json
check trivial comma in EOF before revesing
Revise error json b'm"}]},' ending from comma index -1.
check trivial comma in EOF after revising
b'd": "frfreedom"}]}]}'
G:\py_workspace\ptt\\Gossiping-34700-39632.json parsing start!
parsing finished!
Total iter is 24959
0
2020-03-01 09:04:41.760153
1247
2020-03-01 09:04:48.667202
2494
2020-03-01 09:04:55.799636
3741
2020-03-01 09:05:02.802667
4988
2020-03-01 09:05:09.920579
6235
2020-03-01 09:05:17.350225
7482
2020-03-01 09:05:24.441169
8729
2020-03-01 09:05:31.833940
9976
2020-03-01 09:05:39.429152
11223
2020-03-01 09:05:46.701894
12470
2020-03-01 09:05:53.976732
13717
2020-03-01 09:06:01.251553
14964
2020-03-01 09:06:08.650321
16211
2020-03-01 09:06:16.062099
17458
2020-03-01 09:06:23.990548
18705
2020-03-01 09:06:31.441284
19952
2020-03-01 09:06:38.842252
21199
2020-03-01 09:06:46.138078
22446
2020-03-01 09:06:53.413898
23693
2020-03-01 09:07:00.845997
24940
2020-03-01 09:07:08.261253
HatePolitics-0-4062.json
c

KeyboardInterrupt: 